In [50]:
import pandas as pd
import numpy as np
import torch
import json
import re

In [51]:
reflex={}
with open('iid_question_entity.txt') as f:
    for line in f.readlines():
        reflex[int(line.strip('\n').split('\t')[0])]=line.strip('\n').split('\t')[1]

In [52]:
with open('ke\iid_test.json') as f:
    txt_list=json.load(f)
questions=[]
inputs=[]
title=0
passage_id=0
total=0
for txt in txt_list:
    total+=1
    question={}
    question['id']=txt['id']
    if not title:
        title=txt['title']
    elif txt['title']!=title:
        title=txt['title']
        passage_id+=1
    p=re.compile(r'[[](.*?)[]]', re.S)
    if re.search(p,txt['passage']) is None:
        continue
    desc=re.search(p,txt['passage']).group()[1:-1]
    if desc=='':
        refined=txt['question']['KoRC-H'].replace('[A-Z]',reflex[passage_id])
    elif desc in txt['question']['KoRC-H']:
        refined=txt['question']['KoRC-H'].replace(desc,reflex[passage_id])
    elif ' '.join(desc.split(' ')[1:])=='':
        refined=txt['question']['KoRC-H'].replace('[A-Z]',reflex[passage_id])
    elif ' '.join(desc.split(' ')[1:]) in txt['question']['KoRC-H']:
        refined=txt['question']['KoRC-H'].replace(' '.join(desc.split(' ')[1:]),reflex[passage_id]).replace(' the','').replace(' this','')
    elif desc.replace('[A-Z]','').strip(' ').strip("'s").strip("‘s’").replace(' a ',' ').replace(' the ',' ') in txt['question']['KoRC-H']:
        refined=txt['question']['KoRC-H'].replace(desc.replace('[A-Z]','').strip(' ').strip("'s").strip("‘s’").replace(' a ',' ').replace(' the ',' '),reflex[passage_id]).replace(' the','').replace(' this','')
    elif 'this' in txt['question']['KoRC-H']:
        refined=txt['question']['KoRC-H'].replace('this',reflex[passage_id])
    elif 'the region in North America' in txt['question']['KoRC-H']:
        refined=txt['question']['KoRC-H'].replace('the region in North America',reflex[passage_id])
    elif desc=='of the country in southern Asia':
        refined=txt['question']['KoRC-H'].replace('the Asian country',reflex[passage_id])
    elif desc=='a region in the UK':
        refined=txt['question']['KoRC-H'].replace('region',reflex[passage_id])
    elif desc=='in the state in the western United States on the Pacific coast':
        refined=txt['question']['KoRC-H'].replace('the western United States on the Pacific coast',reflex[passage_id])
    elif desc in ['a constituent country of the United Kingdom',"underdeveloped country's"]:
        refined=txt['question']['KoRC-H'].replace('country',reflex[passage_id])
    elif desc in ["a Nordic Country's","a Nordic country","country rich in mineral resources","a country between the South Pacific and Indian Oceans","country named as the top of Europea","the country of northern North America","a state of the United States","an Asian country"]:
        refined=txt['question']['KoRC-H'].replace('country',reflex[passage_id])
    elif desc.strip("A's") in txt['question']['KoRC-H'].lower():
        refined=txt['question']['KoRC-H'].lower().replace(desc.replace('[A-Z]','').strip("'s"),reflex[passage_id])
    elif desc=='one country A ':
        refined=txt['question']['KoRC-H'].replace('the country A',reflex[passage_id])
    elif desc=='particular country':
        refined=txt['question']['KoRC-H']
    elif desc=='a country A':
        refined=txt['question']['KoRC-H'].replace('the state A',reflex[passage_id])
    elif desc.replace('nation','state') in txt['question']['KoRC-H']:
        refined=txt['question']['KoRC-H'].replace(desc.replace('nation','state'),reflex[passage_id])
    elif desc in ['one of cultural centers in Europe ','a big city','one major industrial city','one of the biggest city','federal capital']:
        refined=txt['question']['KoRC-H'].replace('city',reflex[passage_id])
    elif desc=='one federal republic mainly in North America':
        refined=txt['question']['KoRC-H'].replace('it',reflex[passage_id])
    elif desc=='an independent state':
        refined=txt['question']['KoRC-H'].replace('the state',reflex[passage_id])
    elif desc=='state A of America':
        refined=txt['question']['KoRC-H'].replace('state A',reflex[passage_id])
    elif desc=='sovereign stateT':
        refined=txt['question']['KoRC-H'].replace('sovereign state T',reflex[passage_id])
    elif desc=='province QU‘s':
        refined=txt['question']['KoRC-H'].replace('Province QU',reflex[passage_id])
    elif desc=='sovereign state K':
        refined=txt['question']['KoRC-H'].replace('US territory K',reflex[passage_id])
    elif desc=='sovereign state A':
        refined=txt['question']['KoRC-H'].replace('the sovereign A',reflex[passage_id])       
    elif desc=='a country':
        refined=txt['question']['KoRC-H'].replace('it',reflex[passage_id]).replace('the ocuntry',reflex[passage_id])   
    elif desc=='TV series':
        refined=txt['question']['KoRC-H'].replace('the work',reflex[passage_id]).replace('the TV program',reflex[passage_id])  
    elif desc=="independent state BK's":
        refined=txt['question']['KoRC-H'].replace('Independent state BK',reflex[passage_id])
    elif desc=='independent state N':
        refined=txt['question']['KoRC-H'].replace('an independent country',reflex[passage_id])
    elif desc in ['sovereign nation B','large town B']:
        refined=txt['question']['KoRC-H'].replace('xxx',reflex[passage_id]).replace('XXX',reflex[passage_id])
    else:
        print(txt['question']['KoRC-H'])
        print(desc)
    if refined not in questions:
        question['name']=refined
        questions.append(str(question))
        inputs.append(refined)
    if reflex[passage_id]=='':
        print(txt['passage'])

What territorial entities does the independent stateA consist of?
independent state A


In [42]:
with open('input1.txt','w') as f:
    f.writelines(inputs)

In [53]:
refined_question1=[]
for question in questions:
    refined_question1.append(eval(question))

In [39]:
reflex={}
with open('ood_question_entity.txt') as f:
    for line in f.readlines():
        reflex[int(line.strip('\n').split('\t')[0])]=line.strip('\n').split('\t')[1]

In [40]:
with open('ke\ood_test.json') as f:
    txt_list=json.load(f)
questions=[]
inputs=[]
title=0
passage_id=0
total=0
for txt in txt_list:
    total+=1
    question={}
    question['id']=txt['id']
    if not title:
        title=txt['title']
    elif txt['title']!=title:
        title=txt['title']
        passage_id+=1
    if reflex[passage_id]=='':
        question['name']=txt['question']['KoRC-H']
        questions.append(str(question).replace('   ',' '))
        inputs.append(refined)
        continue        
    p=re.compile(r'[[](.*?)[]]', re.S)
    if re.search(p,txt['passage']) is None:
        continue
    desc=re.search(p,txt['passage']).group()[1:-1]
    if desc=='':
        refined=txt['question']['KoRC-H'].replace('[A-Z]',reflex[passage_id])
    elif desc in txt['question']['KoRC-H']:
        refined=txt['question']['KoRC-H'].replace(desc,reflex[passage_id])
    elif ' '.join(desc.split(' ')[1:])=='':
        refined=txt['question']['KoRC-H'].replace('[A-Z]',reflex[passage_id])
    elif ' '.join(desc.split(' ')[1:]) in txt['question']['KoRC-H']:
        refined=txt['question']['KoRC-H'].replace(' '.join(desc.split(' ')[1:]),reflex[passage_id]).replace(' the','').replace(' this','')
    elif desc.replace('[A-Z]','').strip(' ').strip("'s").strip("‘s’").replace(' a ',' ').replace(' the ',' ') in txt['question']['KoRC-H']:
        refined=txt['question']['KoRC-H'].replace(desc.replace('[A-Z]','').strip(' ').strip("'s").strip("‘s’").replace(' a ',' ').replace(' the ',' '),reflex[passage_id]).replace(' the','').replace(' this','')
    elif 'this' in txt['question']['KoRC-H']:
        refined=txt['question']['KoRC-H'].replace('this',reflex[passage_id])
    elif 'the region in North America' in txt['question']['KoRC-H']:
        refined=txt['question']['KoRC-H'].replace('the region in North America',reflex[passage_id])
    elif desc=='of the country in southern Asia':
        refined=txt['question']['KoRC-H'].replace('the Asian country',reflex[passage_id])
    elif desc=='a region in the UK':
        refined=txt['question']['KoRC-H'].replace('region',reflex[passage_id])
    elif desc=='in the state in the western United States on the Pacific coast':
        refined=txt['question']['KoRC-H'].replace('the western United States on the Pacific coast',reflex[passage_id])
    elif desc in ['a constituent country of the United Kingdom',"underdeveloped country's"]:
        refined=txt['question']['KoRC-H'].replace('country',reflex[passage_id])
    elif desc in ["a Nordic Country's","a Nordic country","country rich in mineral resources","a country between the South Pacific and Indian Oceans","country named as the top of Europea","the country of northern North America","a state of the United States","an Asian country"]:
        refined=txt['question']['KoRC-H'].replace('country',reflex[passage_id])
    elif desc.strip("A's") in txt['question']['KoRC-H'].lower():
        refined=txt['question']['KoRC-H'].lower().replace(desc.replace('[A-Z]','').strip("'s"),reflex[passage_id])
    elif desc=='one country A ':
        refined=txt['question']['KoRC-H'].replace('the country A',reflex[passage_id])
    elif desc=='particular country':
        refined=txt['question']['KoRC-H']
    elif desc=='a country A':
        refined=txt['question']['KoRC-H'].replace('the state A',reflex[passage_id])
    elif desc.replace('nation','state') in txt['question']['KoRC-H']:
        refined=txt['question']['KoRC-H'].replace(desc.replace('nation','state'),reflex[passage_id])
    elif desc in ['one of cultural centers in Europe ','a big city','one major industrial city','one of the biggest city','federal capital']:
        refined=txt['question']['KoRC-H'].replace('city',reflex[passage_id])
    elif desc=='one federal republic mainly in North America':
        refined=txt['question']['KoRC-H'].replace('it',reflex[passage_id])
    elif desc=='an independent state':
        refined=txt['question']['KoRC-H'].replace('the state',reflex[passage_id])
    elif desc=='state A of America':
        refined=txt['question']['KoRC-H'].replace('state A',reflex[passage_id])
    elif desc=='sovereign stateT':
        refined=txt['question']['KoRC-H'].replace('sovereign state T',reflex[passage_id])
    elif desc=='province QU‘s':
        refined=txt['question']['KoRC-H'].replace('Province QU',reflex[passage_id])
    elif desc=='sovereign state K':
        refined=txt['question']['KoRC-H'].replace('US territory K',reflex[passage_id])
    elif desc=='sovereign state A':
        refined=txt['question']['KoRC-H'].replace('the sovereign A',reflex[passage_id])       
    elif desc=='a country':
        refined=txt['question']['KoRC-H'].replace('it',reflex[passage_id]).replace('the ocuntry',reflex[passage_id])   
    elif desc=='TV series':
        refined=txt['question']['KoRC-H'].replace('the work',reflex[passage_id]).replace('the TV program',reflex[passage_id])  
    elif desc=="independent state BK's":
        refined=txt['question']['KoRC-H'].replace('Independent state BK',reflex[passage_id])
    elif desc=='independent state N':
        refined=txt['question']['KoRC-H'].replace('an independent country',reflex[passage_id])
    elif desc in ['sovereign nation B','large town B']:
        refined=txt['question']['KoRC-H'].replace('xxx',reflex[passage_id]).replace('XXX',reflex[passage_id])
    elif desc=='the former leader':
        refined=txt['question']['KoRC-H'].replace('the character',reflex[passage_id])
    elif desc=='Person A':
        refined=txt['question']['KoRC-H'].replace('Person B',reflex[passage_id])
    elif desc=='the city A':
        refined=txt['question']['KoRC-H'].replace('the state A',reflex[passage_id])
    elif desc=='an independent country':
        refined=txt['question']['KoRC-H'].replace('country',reflex[passage_id])
    elif desc=='The man':
        refined=txt['question']['KoRC-H'].replace('the person',reflex[passage_id])
    elif desc=='a man who could handle many style songs':
        refined=txt['question']['KoRC-H'].replace('he',reflex[passage_id]).replace('him',reflex[passage_id])
    elif desc in ['one American iconic figure of 20th-century jazz','the actor']:
        refined=txt['question']['KoRC-H'].replace('he',reflex[passage_id]).replace('his',reflex[passage_id]+"'s")
    elif desc in ['a 3D film','another film','an animated film','a movie','The movie']:
        refined=txt['question']['KoRC-H'].replace('that film',reflex[passage_id]).replace('that moive',reflex[passage_id]).replace('the film',reflex[passage_id]).replace('the movie',reflex[passage_id])
    elif desc=='a famous figure in the entertainment industry':
        refined=txt['question']['KoRC-H'].replace('famous figures in the entertainment industry',reflex[passage_id])
    elif desc=='an Italian musician':
        refined=txt['question']['KoRC-H'].replace('the man',reflex[passage_id]).replace('the artist',reflex[passage_id])
    elif desc=='one network of 6 regional carriers':
        refined=txt['question']['KoRC-H'].replace('the man',reflex[passage_id]).replace('the artist',reflex[passage_id])
    elif desc in ['The famous person','This celebrity']:
        refined=txt['question']['KoRC-H'].replace('he',reflex[passage_id]+"'s").replace('his',reflex[passage_id]+"'s")
    elif desc=='a Finnish singer':
        refined=txt['question']['KoRC-H'].replace('her',reflex[passage_id]+"'s")
    elif desc=='the character in Indian Epics':
        refined=txt['question']['KoRC-H'].replace('the character',reflex[passage_id])
    elif desc=='particular literary work':
        refined=txt['question']['KoRC-H'].replace('the story',reflex[passage_id])
    elif desc=='metropolis A':
        refined=txt['question']['KoRC-H'].replace('xxx',reflex[passage_id])
    elif desc=='General Assembly A':
        refined=txt['question']['KoRC-H'].replace('Plenary session A',reflex[passage_id])
    elif desc=="departement VA's":
        refined=txt['question']['KoRC-H'].replace('Departement VA',reflex[passage_id])
    elif desc=='A video game':
        refined=txt['question']['KoRC-H']
    else:
        print(txt['question']['KoRC-H'])
        print(desc)
    if refined not in questions:
        question['name']=refined
        questions.append(str(question).replace('   ',' '))
        inputs.append(refined+'\n')
total

1853

In [41]:
with open('input2.txt','w') as f:
    f.writelines(inputs)

In [48]:
refined_question2=[]
for question in questions:
    refined_question2.append(eval(question))

## 生成提交文件

In [54]:
lines=[]
with open('predict_iid.json') as fp:
    for line in fp.readlines():
        if line!='\n':
            lines.append(line.replace('\n',''))
result_dict=eval(''.join(lines).replace('  {    ','{').replace(',    ',',').replace('  }{','},{'))

In [55]:
results={}
for i in range(len(result_dict)):
    id,answer=refined_question1[i]['id'],result_dict[i]['answer']
    results[id]=answer
z=json.dumps(results)
with open('iid.json','w') as f:
    json.dump(z,f) 

In [56]:
lines=[]
with open('predict_ood.json') as fp:
    for line in fp.readlines():
        if line!='\n':
            lines.append(line.replace('\n',''))
result_dict=eval(''.join(lines).replace('  {    ','{').replace(',    ',',').replace('  }{','},{'))
results={}
for i in range(len(result_dict)):
    id,answer=refined_question2[i]['id'],result_dict[i]['answer']
    results[id]=answer
z=json.dumps(results)
with open('ood.json','w') as f:
    json.dump(z,f) 